In [11]:
import awkward as ak
import h5py

ModuleNotFoundError: No module named 'awkward'

In [3]:
# load the data from the skimmed file

file='skimmed_ttbarDM.h5'

with h5py.File(file, 'r') as f:
    # Load the form and length metadata
    form = f.attrs['form']
    length = f.attrs['length']
    
    # Load the buffers (numpy arrays) from the HDF5 file
    container = {key: f[key][()] for key in f.keys()}
    
    # Reconstruct the Awkward Array from buffers
    df = ak.from_buffers(form, length, container)
print(ak.__version__)

AttributeError: module 'awkward' has no attribute 'from_buffers'

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from pathlib import Path as Path
import cpt_columnar
from functools import partial
from tqdm import tqdm
import wandb
tqdm = partial(tqdm, position=0, leave=True)
from cpt_columnar import ColumnarCovariantTopFormer
import matplotlib.pylab as pylab
%matplotlib inline
%load_ext autoreload
%autoreload 2

USE_GPU = True
dtype = torch.float32 # we will be using float
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

from seaborn import heatmap
import seaborn as sns
sns.set_style("white")
sns.axes_style("white")

using device: cpu



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/bachjoer/anaconda3/envs/cpt/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/bachjoer/anaconda3/envs/cpt/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/bachjoer/anaconda3/envs/cpt/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/bachjoer/anaconda3/envs/cpt/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in l

{'figure.facecolor': 'white',
 'axes.labelcolor': '.15',
 'xtick.direction': 'out',
 'ytick.direction': 'out',
 'xtick.color': '.15',
 'ytick.color': '.15',
 'axes.axisbelow': True,
 'grid.linestyle': '-',
 'text.color': '.15',
 'font.family': ['sans-serif'],
 'font.sans-serif': ['Arial',
  'DejaVu Sans',
  'Liberation Sans',
  'Bitstream Vera Sans',
  'sans-serif'],
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'image.cmap': 'rocket',
 'xtick.top': False,
 'ytick.right': False,
 'axes.grid': False,
 'axes.facecolor': 'white',
 'axes.edgecolor': '.15',
 'grid.color': '.8',
 'axes.spines.left': True,
 'axes.spines.bottom': True,
 'axes.spines.right': True,
 'axes.spines.top': True,
 'xtick.bottom': False,
 'ytick.left': False}

In [22]:
max_num_output = 2 # assert that there are at most ${max_num_output} tops
base_dir = './trained'
arch = 'ColumnarCovariantTopFormer'
model_params = {
    'in_dim': 9,
    'out_dim': 4,
    'max_num_output': max_num_output,
    'hidden_dim': 256,
    'num_convs': (6, 6),
    'heads': 4,
    'mass': 173,
    'match_scale_factor': torch.FloatTensor([0, 1, 1, 0]), # used for matching dR = (dy, dphi)    
    'p_norm': 2, # used in matching and loss
    'beta': 0.8, # loss weight for predicting number of tops
    'dropout': 0.,
    'schedule_lr': False,
    'use_gpu': USE_GPU,
    'uniform_attention': False,
}
tag = 'TEST' # CHANGE THIS
output_dir = f"{arch}_{model_params['num_convs']}_{model_params['hidden_dim']}_{tag}"
output_dir = os.path.join(base_dir, output_dir)
model_params['output_dir'] = output_dir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model = eval(arch)(**model_params).to(device)

Using Lamb optimizer with lr=0.0001
Initializing a new model from scratch
Model has 9.9M parameters


In [31]:

from torch.utils.data import Dataset, DataLoader

class RandomFloatDataset(Dataset):
    def __init__(self, num_samples, num_features=9):
        self.num_samples = num_samples
        self.num_features = num_features

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Return a random float tensor of size (num_features,)
        return torch.randn(self.num_features)

# Number of samples in the dataset
num_samples = 1000

# Create the dataset
dataset = RandomFloatDataset(num_samples)

# Create the DataLoader
batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example of iterating over the DataLoader
for batch in dataloader:
    print(batch)  # Each batch is a tensor of size (batch_size, num_features)

    break  # Remove this break to see more batches


tensor([[ 0.8567,  1.3374, -0.1243,  ..., -0.0280, -0.5058,  0.8143],
        [ 0.8772, -0.0914, -0.8506,  ..., -0.2863,  0.3045,  0.1952],
        [-0.4699, -0.1087,  0.0434,  ..., -0.9168,  0.1374,  0.0621],
        ...,
        [-1.2165, -0.7947,  0.4149,  ..., -0.3132, -0.8783, -0.5963],
        [ 0.3307, -2.1478,  1.2014,  ..., -0.3847,  1.0297, -0.4359],
        [-0.1710, -0.0950,  1.9840,  ..., -0.6622,  1.1208, -0.5500]])


In [36]:
for batch in dataloader:
    model(batch) 
    
    break  

torch.Size([1, 256, 512])
torch.Size([1, 256, 512])
tensor([[-0.9627, -0.5683, -1.2048,  ...,  0.9909,  2.0373, -1.2303],
        [-0.0220, -0.7441,  1.0866,  ...,  1.5993, -1.5469,  1.0243],
        [ 2.0365, -0.0915, -0.7048,  ...,  0.6779,  0.0210,  0.6760],
        ...,
        [-0.0292,  0.2960, -0.8754,  ...,  1.2320, -0.3449,  0.8475],
        [-0.4934,  0.2437, -1.2067,  ...,  0.7095, -0.2214,  0.1511],
        [-0.3732, -1.1236,  1.0205,  ...,  0.5910,  0.9714,  0.1169]],
       grad_fn=<AddBackward0>)
tensor([[-0.3431, -0.1989,  0.6359,  ...,  1.1756, -0.2253,  0.0421],
        [-0.3431, -0.1989,  0.6359,  ...,  1.1756, -0.2253,  0.0421],
        [-0.3977, -0.6976, -0.7709,  ..., -0.5505,  0.9222, -0.0180],
        ...,
        [-0.0973, -0.2422, -0.0712,  ..., -0.0424,  0.1970,  0.4172],
        [-1.3989,  0.2218,  1.2338,  ..., -0.6529,  0.6559,  0.2337],
        [-1.3989,  0.2218,  1.2338,  ..., -0.6529,  0.6559,  0.2337]],
       grad_fn=<ReshapeAliasBackward0>)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 256 but got size 512 for tensor number 1 in the list.